In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    # duration
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    print('Std Dev of duration', df.describe()['duration']['std'])
    
    # convert to categorical
    df[categorical] = df[categorical].astype(str)
    
    # outliers
    total = len(df)
    outliers_removed_df = df[(df.duration>=1) & (df.duration<=60)].copy()
    no_outliers = len(outliers_removed_df)
    print(f'Total: {total}, After removal: {no_outliers}, Left:{no_outliers/total}')
    return outliers_removed_df

In [3]:
df_jan = read_data('yellow_tripdata_2022-01.parquet')
df_feb = read_data('yellow_tripdata_2022-02.parquet')

Std Dev of duration 46.44530513776499
Total: 2463931, After removal: 2421440, Left:0.9827547930522406
Std Dev of duration 47.26394076781901
Total: 2979431, After removal: 2918187, Left:0.979444397269143


In [4]:
train_dicts = df_jan[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df_jan[target].values

In [5]:
X_train.shape

(2421440, 515)

In [6]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

6.986190834594674

In [7]:
val_dicts = df_feb[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = df_feb[target].values

In [8]:
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.786409124501573